**Connexion au Drive où se trouvent les données**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Chemin d'accès au dossier 'Data' contenant les genres**

In [ ]:
data_path = 'your_path_to_drive/Data/genres_original'
genres = os.listdir(data_path)

**Installation des librairies**

In [ ]:
import librosa
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

!pip install resampy
!pip install pydub

import soundfile as sf
from pydub import AudioSegment

**Extraction des caractéristiques MFCC et préparation des données**

In [ ]:
# Fonction pour extraire les caractéristiques d'un fichier audio
def extract_features(file_path):
    try:
        # Charger le fichier audio
        audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
        # Extraire les MFCC
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        # Moyenne des coefficients MFCC
        mfccs_processed = np.mean(mfccs.T,axis=0)
    except Exception as e:
        print("Erreur rencontrée lors de l'extraction des caractéristiques: ", e)
        return None
    return mfccs_processed

**Préparer les données et les étiquettes**

In [ ]:
features = []
labels = []

**Extraction des caractéristiques pour chaque fichier audio de chaque catégorie musicale**

In [ ]:
for genre in genres:
    # Chemin d'accès aux fichiers pour ce genre
    file_paths = os.listdir(os.path.join(data_path, genre))
    for file in file_paths:
        file_path = os.path.join(data_path, genre, file)
        # Extraire les caractéristiques pour chaque fichier audio
        data = extract_features(file_path)
        if data is not None:
            features.append(data)
            labels.append(genre)
        else:
            print(f"Le fichier {file} a été ignoré en raison d'une erreur de lecture.")

**Conversion de la liste des caractéristiques en un tableau numpy**

In [ ]:
features_array = np.array(features)

**Encodage des étiquettes**

In [ ]:
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)
num_classes = labels_categorical.shape[1]

**Division des données en ensembles d'entraînement et de test**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features_array, labels_categorical, test_size=0.2, random_state=42)

**Redimensionnement des données pour le CNN**

In [ ]:
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

**Construction du Modèle CNN-LSTM pour la Classification des Genres Musicaux**

In [ ]:
# Initialisation d'un modèle séquentiel dans Keras
# Modèle de réseau de neurone
model = Sequential()

# Couches Conv1D
model.add(Conv1D(filters=16, kernel_size=2, activation='relu', input_shape=(X_train_reshaped.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))

# La transition de Conv1D à LSTM nécessite un "aplatissage" temporel, pas un Flatten() qui aplatirait tout
# Le modèle Conv1D retourne déjà des données dans le format attendu par LSTM, donc aucune opération supplémentaire n'est nécessaire

# Couche LSTM
model.add(LSTM(64, return_sequences=False))  # return_sequences=False car la sortie suivante est une couche Dense

# Couche Dense pour la classification
model.add(Dense(num_classes, activation='softmax'))  # num_classes doit être défini en fonction du nombre de genres

# Compiler le modèle
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Afficher le résumé du modèle
model.summary()


**Entraînement du modèle**

In [ ]:
# Callback pour sauvegarder le meilleur modèle basé sur la précision de validation
checkpoint = ModelCheckpoint("best_model.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Callback pour arrêter l'entraînement si aucune amélioration en précision de validation n'est observée après 'patience' époques
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, mode='max')

# Entraîner le modèle
history = model.fit(
    X_train_reshaped,
    y_train,
    epochs=50,  # ajustable pour plus de précsion
    batch_size=32,  # ajustable pour plus de précsion
    validation_split=0.2,  # fraction des données pour la validation
    callbacks=[checkpoint, early_stopping]
)

**Évaluation du modèle**

In [ ]:
# Charger le meilleur modèle sauvegardé
model.load_weights("best_model.hdf5")

# Évaluer le modèle sur l'ensemble de test
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test, verbose=1)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


**Test existence fichier test**

In [ ]:
file_path = 'your_path_to_an_audio_file.mp3'
print(os.path.exists(file_path))

**Transformatioh fichier mp3 en wav**

In [ ]:
file_path_mp3 = 'your_path_to_an_audio_file.mp3'
file_path_wav = 'your_path_to_an_audio_file.wav'

# Convertir mp3 en wav
audio = AudioSegment.from_mp3(file_path_mp3)
audio.export(file_path_wav, format="wav")

# Vérifiez que le fichier wav est bien créé
print(os.path.exists(file_path_wav))


**Prédictions avec le modèle**

In [ ]:
def predict_genre(file_path, model, le):
    # Extraire les caractéristiques du fichier audio
    features = extract_features(file_path)
    if features is not None:
        features_reshaped = features.reshape(1, features.shape[0], 1)

        # Faire la prédiction en utilisant le modèle
        prediction = model.predict(features_reshaped)

        # Convertir la prédiction en nom de genre
        predicted_genre_index = np.argmax(prediction)
        predicted_genre = le.inverse_transform([predicted_genre_index])

        return predicted_genre[0]
    else:
        return "Erreur lors de l'extraction des caractéristiques du fichier audio."

# Utiliser la fonction pour prédire le genre d'un fichier audio
file_path = 'your_path_to_an_audio_file.wav'
genre_pred = predict_genre(file_path, model, le)
print(f"Le genre prédit pour l'extrait musical est : {genre_pred}")



1/1 [==============================] - 0s 473ms/step
Le genre prédit pour l'extrait musical est : metal


[Données utilisées](https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification)

**Rapport**

Au cours de ce projet, nous avons développé un modèle de machine learning pour la classification des genres musicaux. Notre approche s'est basée sur une architecture hybride combinant CNN et LSTM pour traiter efficacement les caractéristiques temporelles et spatiales des signaux audio. Bien que le modèle ait montré une certaine capacité à discriminer entre différents genres, l'accuracy obtenue de 51% indique une marge significative d'amélioration.

Pour améliorer les performances du modèle, nous avons envisagé plusieurs stratégies :

1. Augmentation et Diversification des Données : L'intégration de davantage
d'échantillons musicaux, ainsi que l'application de techniques d'augmentation des données audio, telles que le changement de vitesse et l'ajout de bruit, pour enrichir notre ensemble d'apprentissage et augmenter la robustesse du modèle.

2. Optimisation de l'Architecture du Modèle : Ajustement des couches et des neurones dans les couches Conv1D et LSTM, et expérimentation avec l'ajout de couches de normalisation par lots et de couches de dropout pour une meilleure généralisation.

3. Ajustement Fin des Hyperparamètres : Modification du taux d'apprentissage, de la taille des lots, et du nombre d'époques, en utilisant des méthodes telles que la recherche par grille ou la recherche aléatoire pour trouver la combinaison optimale.

4. Application de Techniques de Régularisation : Mise en œuvre de la régularisation L1/L2 et de l'early stopping pour contrer le surajustement.

5. Évaluation Complète : Utilisation de métriques supplémentaires comme le F1-score et la matrice de confusion pour une évaluation plus nuancée, et application de la validation croisée pour assurer la stabilité et la fiabilité du modèle.

Tout au long du projet, nous avons rencontré des défis tels que l'adaptation des données aux exigences structurelles du modèle et la gestion des fichiers audio mal lus ou corrompus. Ces défis ont été surmontés grâce à un prétraitement méticuleux des données et à une gestion robuste des exceptions lors de l'extraction des caractéristiques.

En conclusion, bien que des améliorations soient nécessaires pour atteindre une précision plus élevée, les stratégies proposées et les leçons tirées de ce projet posent les bases pour de futures itérations et optimisations du modèle.

**Arthur Liot & Paolig Blan**